<a href="https://colab.research.google.com/github/Ddhawan2003/FplAnalytics/blob/main/ScrapingPL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import requests

In [56]:
url = "https://fbref.com/en/comps/9/2022-2023/2022-2023-Premier-League-Stats"

In [57]:
data = requests.get(url)

In [58]:
from bs4 import BeautifulSoup

In [63]:
soup = BeautifulSoup(data.text)

In [65]:
standings_table = soup.select('table.stats_table')[0]

IndexError: list index out of range

In [ ]:
links = standings_table.find_all('a')

In [ ]:
links = [l.get("href") for l in links]

In [ ]:
links = [l for l in links if '/squads' in l]

In [ ]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [ ]:
team_url = team_urls[0]

In [ ]:
data = requests.get(team_url)

In [ ]:
import pandas as pd

In [ ]:
from io import StringIO
matches = pd.read_html(StringIO(data.text), match="Scores & Fixtures")[0]

In [ ]:
soup = BeautifulSoup(data.text)

In [ ]:
links = soup.find_all('a')

In [ ]:
links = [l.get("href") for l in links]

In [ ]:
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [ ]:
data = requests.get(f"https://fbref.com{links[0]}")

In [ ]:
shooting = pd.read_html(StringIO(data.text), match="Shooting")[0]

In [ ]:
shooting.columns = shooting.columns.droplevel()

In [ ]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [ ]:
shooting.shape

(62, 26)

In [ ]:
team_data.shape

(61, 25)

In [49]:
years = list(range(2024, 2020, -1))

In [50]:
all_matches = []

In [51]:
standings_url = "https://fbref.com/en/comps/9/2023-2024/2023-2024-Premier-League-Stats"

In [54]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(StringIO(data.text), match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(StringIO(data.text), match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]

        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)

IndexError: list index out of range